# Непараметрическая регрессия  

Задачу восстановления регрессии можно определить следующим образом: известно множество объектов (обучающая выборка), и необходимо построить функцию, лучше всего её приближающую.  
Как и в задаче классификации, обучающая выборка *X* представляет собой матрицу объектов-признаков, но множество ответов *Y* - всё множество вещественных чисел.

В качестве функционала качества будем использовать сумму квадратов ошибок: ![](https://latex.codecogs.com/gif.latex?Q%3D%5Csum_%7Bi%3D1%7D%5El%20%28%5Cphi%28x_i%20%2C%5Calpha%29-y_i%29%5E2), где ![](https://latex.codecogs.com/gif.latex?%5Cphi%28x_i%20%2C%5Calpha%29) - модель регрессии, ![](https://latex.codecogs.com/gif.latex?%5Calpha) - функция регрессии.

В непараметрической регрессии функция регрессии для объекта *x* строится по нескольким ближайшим к нему объектам (степень близости определяется метрикой).

Расмотрим костантную модель регрессии: ![](https://latex.codecogs.com/gif.latex?%5Cphi%28x%2C%5Calpha%29%3D%5Calpha).  
Каждому объекту обучающей выборке присвоим некоторый вес. В качестве весовой функции возьмём функцию ядра (невозрастающую, гладкую, ограниченную функцию): ![](https://latex.codecogs.com/gif.latex?w_i%28x%29%3DK%28%5Cdfrac%7B%5Crho%28x%2Cx_i%29%7D%7Bh%7D%29), где *h* - ширина окна.  
Будем искать оптимальные параметры для ![](https://latex.codecogs.com/gif.latex?%5Calpha), минимизируя ![](https://latex.codecogs.com/gif.latex?Q%28%5Calpha%2CX%5El%29%3D%5Csum_%7Bi%3D1%7D%5Elw_i%28x%29%28%5Calpha%20-%20y_i%29%5E2).

![](https://latex.codecogs.com/gif.latex?%5Calpha_h%28x%2CX%5El%29) будем вычислять по формуле: ![](https://latex.codecogs.com/gif.latex?%5Calpha_h%28x%2CX%5El%29%3D%5Cdfrac%7B%5Csum_%7Bi%3D1%7D%5Ely_iw_i%28x%29%7D%7B%5Csum_%7Bi%3D1%7D%5Elw_i%28x%29%7D%3D%5Cdfrac%7B%5Csum_%7Bi%3D1%7D%5Ely_iK%28%5Cdfrac%7B%5Crho%28x%2Cx_i%29%7D%7Bh%7D%29%7D%7B%5Csum_%7Bi%3D1%7D%5ElK%28%5Cdfrac%7B%5Crho%28x%2Cx_i%29%7D%7Bh%7D%29%7D).

In [4]:
from sklearn.datasets import load_iris
from matplotlib import pyplot as plt
import numpy as np


data = load_iris()['data']
target = load_iris()['target']


[[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]
 [5.4 3.9 1.7 0.4]
 [4.6 3.4 1.4 0.3]
 [5.  3.4 1.5 0.2]
 [4.4 2.9 1.4 0.2]
 [4.9 3.1 1.5 0.1]
 [5.4 3.7 1.5 0.2]
 [4.8 3.4 1.6 0.2]
 [4.8 3.  1.4 0.1]
 [4.3 3.  1.1 0.1]
 [5.8 4.  1.2 0.2]
 [5.7 4.4 1.5 0.4]
 [5.4 3.9 1.3 0.4]
 [5.1 3.5 1.4 0.3]
 [5.7 3.8 1.7 0.3]
 [5.1 3.8 1.5 0.3]
 [5.4 3.4 1.7 0.2]
 [5.1 3.7 1.5 0.4]
 [4.6 3.6 1.  0.2]
 [5.1 3.3 1.7 0.5]
 [4.8 3.4 1.9 0.2]
 [5.  3.  1.6 0.2]
 [5.  3.4 1.6 0.4]
 [5.2 3.5 1.5 0.2]
 [5.2 3.4 1.4 0.2]
 [4.7 3.2 1.6 0.2]
 [4.8 3.1 1.6 0.2]
 [5.4 3.4 1.5 0.4]
 [5.2 4.1 1.5 0.1]
 [5.5 4.2 1.4 0.2]
 [4.9 3.1 1.5 0.2]
 [5.  3.2 1.2 0.2]
 [5.5 3.5 1.3 0.2]
 [4.9 3.6 1.4 0.1]
 [4.4 3.  1.3 0.2]
 [5.1 3.4 1.5 0.2]
 [5.  3.5 1.3 0.3]
 [4.5 2.3 1.3 0.3]
 [4.4 3.2 1.3 0.2]
 [5.  3.5 1.6 0.6]
 [5.1 3.8 1.9 0.4]
 [4.8 3.  1.4 0.3]
 [5.1 3.8 1.6 0.2]
 [4.6 3.2 1.4 0.2]
 [5.3 3.7 1.5 0.2]
 [5.  3.3 1.4 0.2]
 [7.  3.2 4.7 1.4]
 [6.4 3.2 4.5 1.5]
 [6.9 3.1 4.